# Ερώτηση 4

In [1]:
import pandas as pd
import numpy as np
import json
import time 

dataset_path = '../yelp-dataset/' # Change this path to match your local dataset folder path

business_filename = 'yelp_academic_dataset_business.json'
users_filename = 'yelp_academic_dataset_user.json'
review_filename = 'yelp_academic_dataset_review.json'

In [2]:
# Loading total businesses with at least 50 reviews
business_list = []
with open(dataset_path + business_filename, 'r', encoding="utf8") as buisness_file:
    for line in buisness_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 50:
            business_list.append([ json_dict['business_id'], 
                                   json_dict['name'], 
                                   json_dict['review_count']]) 

In [3]:
business_list = np.array(business_list)

In [4]:
# Businesses that have at least 50 reviews
B = set(business_list[:,0])
len(B)

31942

In [5]:
# Business-user ids pairs
start = time.time()

reviews_list = []
with open(dataset_path + review_filename, 'r', encoding="utf8") as reviews_file:
    for line in reviews_file:
        json_dict = json.loads(line)
        if json_dict['business_id'] in B:
            reviews_list.append([json_dict['business_id'], json_dict['user_id']])

reviews_list = np.array(reviews_list)       
stop = time.time()
print("Read file in {:.3f} mins".format((stop-start)/60))

Read file in 1.462 mins


In [6]:
# Conting unique users that commented at least 20 times on B

users = reviews_list[:,1]
users, counts = np.unique(users, return_counts=True)
unique_users = []
for i in range(len(users)):
    if counts[i] >= 20:
        unique_users.append(users[i])
unique_users = np.array(unique_users)
len(unique_users)

40225

In [7]:
# convert to set for efficiency
U = set(unique_users)

In [16]:
start = time.time()

list_users = []
with open(dataset_path + users_filename, 'r', encoding="utf8") as users_file:
    for line in users_file:
        json_dict = json.loads(line)
        if json_dict['user_id'] in U:
            list_users.append([ json_dict['user_id'], 
                                json_dict['friends'].replace(',', '').split()])
stop = time.time()
print("Read file in {:.3f} mins".format((stop-start)/60))

Read file in 0.704 mins


In [17]:
len(list_users)

40225

In [31]:
list_users[0]

['ntlvfPzc8eglqvk92iDIAw',
 ['oeMvJh94PiGQnx_6GlndPQ',
  'wm1z1PaJKvHgSDRKfwhfDg',
  'IkRib6Xs91PPW7pon7VVig',
  'A8Aq8f0-XvLBcyMk2GJdJQ',
  'eEZM1kogR7eL4GOBZyPvBA',
  'e1o1LN7ez5ckCpQeAab4iw',
  '_HrJVzFaRFUhPva8cwBjpQ',
  'pZeGZGzX-ROT_D5lam5uNg',
  '0S6EI51ej5J7dgYz3-O0lA',
  'woDt8raW-AorxQM_tIE2eA',
  'hWUnSE5gKXNe7bDc8uAG9A',
  'c_3LDSO2RHwZ94_Q6j_O7w',
  '-uv1wDiaplY6eXXS0VwQiA',
  'QFjqxXn3acDC7hckFGUKMg',
  'ErOqapICmHPTN8YobZIcfQ',
  'mJLRvqLOKhqEdkgt9iEaCQ',
  'VKX7jlScJSA-ja5hYRw12Q',
  'ijIC9w5PRcj3dWVlanjZeg',
  'CIZGlEw-Bp0rmkP8M6yQ9Q',
  'OC6fT5WZ8EU7tEVJ3bzPBQ',
  'UZSDGTDpycDzrlfUlyw2dQ',
  'deL6e_z9xqZTIODKqnvRXQ',
  '5mG2ENw2PylIWElqHSMGqg',
  'Uh5Kug2fvDd51RYmsNZkGg',
  '4dI4uoShugD9z84fYupelQ',
  'EQpFHqGT9Tk6YSwORTtwpg',
  'o4EGL2-ICGmRJzJ3GxB-vw',
  's8gK7sdVzJcYKcPv2dkZXw',
  'vOYVZgb_GVe-kdtjQwSUHw',
  'wBbjgHsrKr7BsPBrQwJf2w',
  'p59u2EC_qcmCmLeX1jCi5Q',
  'VSAZI1eHDrOPRWMK4Q2DIQ',
  'efMfeI_dkhpeGykaRJqxfQ',
  'x6qYcQ8_i0mMDzSLsFCbZg',
  'K_zSmtNGw1fu-vmxyT

## Creating G graph

In [36]:
import networkx as nx

G = nx.Graph()
for user in list_users:
    for friend in user[1]: # user friendlist
        if friend in U: # don't add users that aren't in U.
            G.add_edge(user[0], friend)

In [37]:
len(G.nodes())

31983

In [40]:
largest_cc = max(nx.connected_components(G), key=len)
CC_max = nx.subgraph(G, largest_cc)

In [43]:
len(CC_max.nodes())

31815